# Урок 2. Парсинг HTML. BeautifulSoup, MongoDB

Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

    *Наименование вакансии
    *Предлагаемую зарплату (отдельно мин. и отдельно макс.)
    *Ссылку на саму вакансию        
    *Сайт откуда собрана вакансия
По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [1]:
from bs4 import BeautifulSoup as bs
import requests as req
import pandas as pd
from random import randint as rndi
from time import sleep

In [2]:
pd.set_option('display.max_colwidth', -1)

#### собираем с hh мин, макс зп и валюту зп

In [3]:
def get_hh_compensations(item):
    try:
        vac_compensation = item.find('div', {'class':'vacancy-serp-item__compensation'}).getText()
    except:
        vac_compensation = 'None'
    if vac_compensation != 'None':
        vac_compensation = vac_compensation.split(' ')
        comp_curr = vac_compensation[-1]
        if vac_compensation[0] == 'от':
            min_compensation = ''.join(vac_compensation[1:-1])
            max_compensation = 'None'
        elif vac_compensation[0] == 'до':
            max_compensation = ''.join(vac_compensation[1:-1])
            min_compensation = 'None'
        else:
            min_compensation = vac_compensation[0].split('-')[0]
            max_compensation = vac_compensation[0].split('-')[1]   
    else:
        min_compensation, max_compensation, comp_curr = ['None'] * 3
    return min_compensation, max_compensation, comp_curr

#### собираем с superjob мин, макс зп и валюту зп

In [4]:
def get_sj_compensations(item):
    try:
        vac_compensation = item.find('span', {'class':'_3mfro _2Wp8I f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'}).getText()
    except:
        vac_compensation = 'None'
    if vac_compensation != 'None':
        if vac_compensation == 'По договорённости':
            min_compensation = 'None'
            max_compensation = 'None'
            comp_curr = 'None'
        elif '—' in vac_compensation:
            vac_compensation = vac_compensation.split('—')
            comp_curr = vac_compensation[1][-1]
            max_compensation = vac_compensation[1][:-1]
            min_compensation = vac_compensation[0]
        elif 'от' in vac_compensation:
            vac_compensation = vac_compensation.split()
            comp_curr = vac_compensation[-1]
            max_compensation = 'None'
            min_compensation = ' '.join(vac_compensation[1:-1])
        elif 'до' in vac_compensation:
            vac_compensation = vac_compensation.split()
            comp_curr = vac_compensation[-1]
            max_compensation = ' '.join(vac_compensation[1:-1])
            min_compensation = 'None'
        else:
            vac_compensation = vac_compensation.split()
            comp_curr = vac_compensation[-1]
            max_compensation = ' '.join(vac_compensation[:-1])
            min_compensation = ' '.join(vac_compensation[:-1])
    else:
        min_compensation, max_compensation, comp_curr = ['None'] * 3
    return min_compensation, max_compensation, comp_curr

#### собираем с hh остальные данные. vac_name - название вакансии, vac_link - ссылка, employer- работодатель, location - местоположение

In [5]:
def get_hh_data(vacancies):
    for item in vacancies:
        site_name = hh
        try:
            vac_name = item.find('a').getText()
        except:
            vac_name = 'None'
        try:    
            vac_link = item.find('a')['href']
        except:
            vac_link = 'None'
        try:
            employer = item.find('a', {'data-qa':'vacancy-serp__vacancy-employer'}).getText()
        except:
            employer = 'None'
        try:    
            location = item.find('span', {'data-qa':'vacancy-serp__vacancy-address'}).getText()
        except:
            location = 'None'
        min_compensation, max_compensation, comp_curr = get_hh_compensations(item)
        df.loc[len(df) + 1] = [vac_name, employer, location, min_compensation, max_compensation, 
                                comp_curr, vac_link, site_name]

#### собираем с superjob остальные данные. vac_name - название вакансии, vac_link - ссылка, employer- работодатель, location - местоположение

In [6]:
def get_sj_data(vacancies):
    for item in vacancies:
        site_name = sj
        try:
            vacancy = item.find('div', {'class':'_3mfro CuJz5 PlM3e _2JVkc _3LJqf'})
            vac_name = vacancy.getText()
        except:
            vac_name = 'None'
        try:
            vac_link = sj + vacancy.findParent('a')['href']
        except:
            vac_link = 'None'
        try:
            employer = item.find('span', {'class':'_3mfro _3Fsn4 f-test-text-vacancy-item-company-name _9fXTd _2JVkc _3e53o _15msI'}).getText()          
        except:
            employer = 'None'
        try:
            location = item.find('span', {'class':'_3mfro f-test-text-company-item-location _9fXTd _2JVkc _3e53o'}).findChildren(limit=2)[1].getText()
        except:
            location = 'None'
        min_compensation, max_compensation, comp_curr = get_sj_compensations(item)
        df.loc[len(df) + 1] = [vac_name, employer, location, min_compensation, max_compensation, 
                                comp_curr, vac_link, site_name]

In [7]:
df = pd.DataFrame(columns=['vac_name', 'employer', 'location', 'min_compensation', 'max_compensation',
                          'currency', 'link', 'site_name'])

In [8]:
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) \
Chrome/78.0.3904.97 Safari/537.36'
headers = {'User-Agent': user_agent}
hh = 'https://hh.ru'
sj = 'https://www.superjob.ru'

#### фомируем строку для поиска

In [9]:
search_text = input('Что ищем? ')
if search_text == '':
    search_text = 'Python'

Что ищем? грузчик


#### берем с hh количество вакансий и страниц по нашему запросу

In [10]:
response = req.get(hh + f'/search/vacancy?text={search_text}', headers=headers)
if response.ok:
    hh_parsed_html = bs(response.text,'lxml')
    try:
        hh_vacancies_total = hh_parsed_html.find('div', {'class':'breadcrumbs'}).next_sibling.getText()
    except:
        hh_vacancies_total = 0
    try:    
        hh_pages_total = int(hh_parsed_html.find_all('a', {'data-qa':'pager-page'})[-1].getText())
    except:
        hh_pages_total = 0
else:
    print('error', response.status_code)

In [11]:
print(f'на {hh} найдено {hh_vacancies_total}. всего страниц {hh_pages_total}')
input_pages = input('Сколько страниц будем парсить? ')
try:
    hh_pages = int(input_pages)
except:
    hh_pages = 0
if hh_pages > hh_pages_total:
    hh_pages = hh_pages_total
elif hh_pages < 1:
        hh_pages = 0

на https://hh.ru найдено 7 058 вакансий «грузчик». всего страниц 100
Сколько страниц будем парсить? 4


#### берем с superjob количество вакансий и страниц по нашему запросу

In [12]:
response = req.get(sj + f'/vacancy/search/?keywords={search_text}&geo%5Bc%5D%5B0%5D=1', headers=headers)
if response.ok:
    sj_parsed_html = bs(response.text,'lxml')
    try:
        sj_vacancies_total = sj_parsed_html.find('div', {'class':'_1tH7S _1o0Xp GPKTZ _3achh _3ofxL _2_FIo'}).findChild().getText()
    except:
        sj_vacancies_total = 0
    if ('Вакансий не найдено' in sj_vacancies_total) or (sj_vacancies_total == 0):
        sj_pages_total = 0
    else:
        try:
            sj_pages_total = int(sj_parsed_html.find_all('span', {'class':'qTHqo _2h9me DYJ1Y _2FQ5q _2GT-y'})[-2].getText())
        except:
            sj_pages_total = 1
else:
    print('error', response.status_code)

In [13]:
print(f'на {sj} {sj_vacancies_total} "{search_text}". всего страниц {sj_pages_total}')
input_pages = input('Сколько страниц будем парсить? ')
try:
    sj_pages = int(input_pages)
except:
    sj_pages = 0
if sj_pages > sj_pages_total:
        sj_pages = sj_pages_total
elif sj_pages < 1:
        sj_pages = 0

на https://www.superjob.ru Найдена 32 581 вакансия "грузчик". всего страниц 61
Сколько страниц будем парсить? 3


#### ходим по hh и собираем данные

In [14]:
for page in range(hh_pages):
    response = req.get(hh + f'/search/vacancy?text={search_text}&page={page}', headers=headers)
    if response.ok:
        hh_parsed_html = bs(response.text,'lxml')
        hh_vacancies_block = hh_parsed_html.find('div', {'class':'vacancy-serp'})
        hh_vacancies = hh_vacancies_block.findChildren('div', {'class':'vacancy-serp-item'}, recursive=False)
        get_hh_data(hh_vacancies)
    else:
        continue  
    sleep(rndi(1, 5))    

#### ходим по superjob и собираем данные

In [15]:
for page in range(sj_pages):
    response = req.get(sj + f'/vacancy/search/?keywords={search_text}&geo%5Bc%5D%5B0%5D=1&page={page + 1}', 
                   headers=headers)
    if response.ok:
        sj_parsed_html = bs(response.text,'lxml')
        sj_vacancies = sj_parsed_html.find_all('div', {'class':'_3zucV _2GPIV f-test-vacancy-item i6-sc _3VcZr'})
        get_sj_data(sj_vacancies)
    else:
        continue
    sleep(rndi(1, 5))

#### вывод результатов

In [16]:
df

,vac_name,employer,location,min_compensation,max_compensation,currency,link,site_name
1,Грузчик-комплектовщик,ООО Транс Альянс Логистик-А,"Москва, Тверская и еще 2",60 000,None,руб.,https://stary-oskol.hh.ru/vacancy/34618992?query=%D0%B3%D1%80%D1%83%D0%B7%D1%87%D0%B8%D0%BA,https://hh.ru
2,Кладовщик/грузчик-комплектовщик,ООО Стант-Креп,Гомель,900,None,бел. руб.,https://stary-oskol.hh.ru/vacancy/34630757?query=%D0%B3%D1%80%D1%83%D0%B7%D1%87%D0%B8%D0%BA,https://hh.ru
3,Грузчик на склад продуктов питания ( Лобня),ВкусВилл,Лобня,36 000,None,руб.,https://stary-oskol.hh.ru/vacancy/33456680?query=%D0%B3%D1%80%D1%83%D0%B7%D1%87%D0%B8%D0%BA,https://hh.ru
4,Грузчик,ТОО Lam.Tech.,Нур-Султан,100 000,100 000,KZT,https://stary-oskol.hh.ru/vacancy/34619909?query=%D0%B3%D1%80%D1%83%D0%B7%D1%87%D0%B8%D0%BA,https://hh.ru
5,Кладовщик-грузчик,Возим-Бай,"Минск, Пролетарская",700,1 000,бел. руб.,https://stary-oskol.hh.ru/vacancy/34623103?query=%D0%B3%D1%80%D1%83%D0%B7%D1%87%D0%B8%D0%BA,https://hh.ru
6,Грузчик-экспедитор (хутор Октябрьский),Связной,Краснодар,35 000,None,руб.,https://stary-oskol.hh.ru/vacancy/34475608?query=%D0%B3%D1%80%D1%83%D0%B7%D1%87%D0%B8%D0%BA,https://hh.ru
7,Грузчик,Унитарное предприятие «Кока-Кола Бевриджиз Белоруссия».,Минск,900,None,бел. руб.,https://stary-oskol.hh.ru/vacancy/34260513?query=%D0%B3%D1%80%D1%83%D0%B7%D1%87%D0%B8%D0%BA,https://hh.ru
8,Заведующий складом,ООО ТРЕЙДПАК,"Москва, Озёрная",60 000,70 000,руб.,https://stary-oskol.hh.ru/vacancy/33654055?query=%D0%B3%D1%80%D1%83%D0%B7%D1%87%D0%B8%D0%BA,https://hh.ru
9,Грузчик-комплектовщик,ООО Доминоспицца,Минск,600,700,бел. руб.,https://stary-oskol.hh.ru/vacancy/32085456?query=%D0%B3%D1%80%D1%83%D0%B7%D1%87%D0%B8%D0%BA,https://hh.ru
10,Ночной Грузчик-комплектовщик,Унитарное предприятие «Кока-Кола Бевриджиз Белоруссия».,Минск,800,1 800,бел. руб.,https://stary-oskol.hh.ru/vacancy/34211697?query=%D0%B3%D1%80%D1%83%D0%B7%D1%87%D0%B8%D0%BA,https://hh.ru
